In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import csv
import time
import os

def get_news(driver):
    # 기사 크롤링
    fieldnames = ["title", "thumbnail", "short_content", "write_time", "link", "company"]
    data = []

    for i in range(1, 11):
        # 페이지 열기
        url = f'https://www.kmib.co.kr/article/list.asp?sid1=all&sid2=&page={i}'
        driver.get(url)
        time.sleep(2)

        articles = driver.find_elements(By.CSS_SELECTOR, '.nws_list.flex.flex_fw.flex_aifs.rgap_lg6.rgap_xs4 > div')

        for article in articles:
            try:
                # 제목(title)
                title_element = article.find_element(By.CSS_SELECTOR, '.tit.fs_lg20.fs_xs16.fw_lgmd.lh_lg15 > a')
                title = title_element.text.strip()

                # 썸네일(thumbnail)
                thumbnail_element = article.find_elements(By.CSS_SELECTOR, 'figure > a > img')
                thumbnail = thumbnail_element[0].get_attribute('src').strip() if thumbnail_element else None

                # 요약 내용(short_content)
                short_content_element = article.find_elements(By.CSS_SELECTOR, '.card_body.rgap_lg1 > p')
                short_content = short_content_element[0].text.strip() if short_content_element else None

                # 작성 시간(write_time)
                write_time_element = article.find_elements(By.CSS_SELECTOR, '.date.gray500.fs_lg14.fw_lgrg')
                write_time = write_time_element[0].text.strip() if write_time_element else None

                # 기사 링크(link) - **href**로 수정
                link = title_element.get_attribute('href').strip()

                # 데이터 추가
                data.append({
                    "title": title,
                    "thumbnail": thumbnail,
                    "short_content": short_content,
                    "write_time": write_time,
                    "link": link,
                    "company": "국민일보"
                })

            except Exception as e:
                print(f"데이터 수집 중 오류 발생: {e}")
                continue

    # CSV 파일로 저장
    save_to_csv(data, fieldnames)

    driver.quit()

def save_to_csv(data, fieldnames):
    # CSV 파일 저장
    folder_path = './Completed_csv'
    
    # 폴더가 없으면 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"{folder_path} 폴더가 생성되었습니다.")

    # 파일 경로 설정
    filename = os.path.join(folder_path, 'Gookmin.csv')
    
    # CSV 파일 저장
    try:
        with open(filename, mode='w', newline='', encoding='utf-8-sig') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()  # 헤더 작성
            for item in data:
                writer.writerow(item)
        print(f"데이터가 저장되었습니다.")
    except Exception as e:
        print(f"CSV 저장 중 오류 발생: {e}")

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

def main():
    options = Options()
    options.add_argument('--headless')  # GUI 없이 실행
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    # WebDriverManager를 사용하여 ChromeDriver 자동 관리
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    print("ChromeDriver 실행 완료")

    get_news(driver)

    driver.quit()

if __name__ == '__main__':
    main()


ChromeDriver 실행 완료
데이터가 저장되었습니다.
